In [1]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ripe.atlas.sagan import PingResult
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from luminol.anomaly_detector import AnomalyDetector

import warnings
warnings.filterwarnings('ignore')

In [13]:
# Import data
measurement_data = "measurement_data/ping-measurement_1.json"

with open(measurement_data) as f:
    data = json.load(f)
    results = [PingResult(result) for result in data]
    del data
    results_dict = []
    # for i in range(1000):
    for result in results:
        # create dictionary from result object

        # result_dict = results[i].__dict__
        result_dict = result.__dict__
        # remove packets key and make keys for each packet in packets
        # packets = result_dict.pop('packets')
        # for i in range(len(packets)):
        #     packet_dict: dict = packets[i].__dict__
        #     for key, value in packet_dict.items():
        #         result_dict[f'packet_{i+1} {key}'] = value
        results_dict.append(result_dict)
    df: pd.DataFrame = pd.DataFrame(results_dict)

In [14]:
df["dropped_packets"] = df["packets_sent"] - df["packets_received"]
df = df.drop(["_on_error", "is_error", "seconds_since_sync", "_on_malformation", "is_malformed", 
              "raw_data","error_message", "measurement_id", "firmware", "group_id", 
              "af", "bundle", "step", "packets", "protocol", "origin", 
              "destination_name", "destination_address", "packets_sent", 
              "packets_received", "packet_size"], axis=1)
df = df.dropna()
df

,created,probe_id,duplicates,rtt_average,rtt_max,rtt_min,rtt_median,dropped_packets
0,2021-06-07 00:02:11+00:00,6025,0,149.218,149.233853,149.195256,149.225,0
1,2021-06-07 00:02:21+00:00,6030,0,166.576,166.850203,166.409528,166.467,0
2,2021-06-07 00:02:22+00:00,6031,0,149.319,149.741417,149.048305,149.168,0
3,2021-06-07 00:02:21+00:00,6039,0,170.098,170.111447,170.077631,170.105,0
4,2021-06-07 00:02:17+00:00,6041,0,173.648,173.714002,173.588153,173.642,0
...,...,...,...,...,...,...,...,...
678089,2021-06-09 23:58:21+00:00,7006,0,147.874,147.964410,147.811359,147.845,0
678090,2021-06-09 23:58:17+00:00,7007,0,208.156,208.185379,208.128699,208.153,0
678091,2021-06-09 23:58:09+00:00,7008,0,162.362,162.421941,162.276314,162.388,0
678092,2021-06-09 23:58:08+00:00,7009,0,219.927,219.953030,219.913387,219.915,0


In [15]:
# Check amount of data points per probe
df["probe_id"].value_counts()

6278    3038
6289    2083
6215    1509
6025    1080
6670    1080
        ... 
7009     725
6834     677
6583     535
6672     179
7010     124
Name: probe_id, Length: 592, dtype: int64

In [65]:
# Add row with difference to median for rtt
median_dict = {}
for i in df["probe_id"].unique():
    single_probe = df[df["probe_id"] == i]
    median_dict[i] = single_probe["rtt_min"].median()

def find_difference(probe_id, rtt_average):
    return rtt_average - median_dict[probe_id]

df["rtt_min_"] = np.vectorize(find_difference)(df['probe_id'], df['rtt_average'])

In [19]:
df = df.reset_index()
df.to_feather("measurement_data/ping-measurement_1.feather")